# Predicción de Default en Préstamos — Proyecto

Este notebook replica la estructura del original y completa un ETL + EDA básicos sobre el sample de Lending Club. Se incluyen justificaciones, tabla de tipos, manejo de NaNs, guardado/carga (Pickle y JSON) y un pipeline reproducible.

## Descripción y objetivo
- Datos: sample de Lending Club (2007–2017Q3), ~100k filas, ~150 columnas.
- Objetivo: realizar un ETL para dejar datos en formato ‘tidy’ y preparar un EDA mínimo.
- Entregables en este notebook: lectura robusta, tabla `(column_name, type)`, conversión de tipos, manejo de NaNs/imputación, guardado/carga de `datos_dict` (Pickle), y JSON de estrategias de imputación.


In [ ]:
import pandas as pd
import numpy as np
import json
from pathlib import Path

pd.options.display.max_columns = 120
pd.options.display.width = 140

DATA_URL = 'https://github.com/sonder-art/fdd_prim_2023/blob/main/codigo/pandas/LoansData_sample.csv.gz?raw=true'
DICT_URL = 'https://resources.lendingclub.com/LCDataDictionary.xlsx'


## ETL — Lectura de datos
La lectura directa del CSV comprimido puede arrojar errores típicos (compresión, líneas problemáticas). Se resuelve con argumentos adicionales de `read_csv` (compression, engine, on_bad_lines, low_memory).


In [ ]:
def read_loans(url: str = DATA_URL) -> pd.DataFrame:
    # Manejo de compresión explícito y líneas problemáticas.
    # Tip: dependiendo de la versión de pandas, 'compression="infer"' también funciona.
    df = pd.read_csv(
        url,
        compression='gzip',
        low_memory=False,
        engine='python',
        on_bad_lines='skip'
    )
    return df

loans_raw = read_loans(DATA_URL)
loans_raw.shape


## Tabla (column_name, type) — estado inicial
Justificación breve: esta tabla ayuda a decidir conversions (porcentajes → float, fechas → datetime, categorías → category, etc.).


In [ ]:
def build_column_types(df: pd.DataFrame) -> pd.DataFrame:
    ct = df.dtypes.reset_index().rename(columns={'index': 'column_name', 0: 'type'})
    ct['type'] = ct['type'].astype(str)
    return ct

column_types_initial = build_column_types(loans_raw)
column_types_initial.head()


## Cargar descripción de columnas (Data Dictionary)
Se utiliza el diccionario público del dataset y se guarda/carga en Pickle.


In [ ]:
datos_dict = pd.read_excel(DICT_URL)
datos_dict.columns = ['feature', 'description']
datos_dict.head(3)


### Pickle — guardar y cargar
Se persiste el diccionario con Pickle para reutilizarlo localmente.


In [ ]:
# Guardar
pickle_path = Path('datos_dict.pkl')
datos_dict.to_pickle(pickle_path)
# Cargar
datos_dict2 = pd.read_pickle(pickle_path)
assert datos_dict2.equals(datos_dict)
datos_dict2.shape


## Conversión de tipos (ETL)
Decisiones:
- `int_rate`, `revol_util`: strings con `%` → float en [0,1].
- `term`: '36 months' → 36 (int).
- `issue_d`, `earliest_cr_line`: `%b-%Y` → datetime.
- `emp_length`: mapeo a años (0–10).
- Categóricas: `home_ownership`, `purpose`, `grade`, `sub_grade`, `verification_status`, `loan_status`.


In [ ]:
def coerce_types(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    # Fechas
    for col, fmt in [('issue_d', '%b-%Y'), ('earliest_cr_line', '%b-%Y')]:
        if col in out.columns:
            out[col] = pd.to_datetime(out[col], format=fmt, errors='coerce')
    # Porcentajes a fracción
    for col in ['int_rate', 'revol_util']:
        if col in out.columns:
            out[col] = pd.to_numeric(out[col].astype(str).str.rstrip('%'), errors='coerce') / 100.0
    # Term en meses
    if 'term' in out.columns:
        out['term'] = pd.to_numeric(out['term'].astype(str).str.extract(r'(\d+)')[0], errors='coerce')
    # Años de empleo
    if 'emp_length' in out.columns:
        mapping = {
            '10+ years': 10, '9 years': 9, '8 years': 8, '7 years': 7, '6 years': 6,
            '5 years': 5, '4 years': 4, '3 years': 3, '2 years': 2, '1 year': 1,
            '< 1 year': 0, 'n/a': np.nan, 'NaN': np.nan
        }
        out['emp_length'] = pd.to_numeric(out['emp_length'].replace(mapping), errors='coerce')
    # Categóricas
    for catcol in ['home_ownership','purpose','grade','sub_grade','verification_status','loan_status']:
        if catcol in out.columns:
            out[catcol] = out[catcol].astype('category')
    return out

loans_typed = coerce_types(loans_raw)
build_column_types(loans_typed).head()


## Manejo de NaNs o missings
Estrategia simple y justificable:
- Numéricas: imputar con mediana (robusta a outliers).
- Fechas: imputar con la moda (valor más frecuente).
- Categóricas/objetos: usar identificador `'missing'` (permite rastrear imputaciones).
Se registra todo en un JSON `{col: {estrategia, valor}}` para reproducibilidad.


In [ ]:
def build_imputation_strategies(df: pd.DataFrame) -> dict:
    strategies = {}
    for col in df.columns:
        s = df[col]
        if pd.api.types.is_numeric_dtype(s):
            val = float(s.median()) if s.notna().any() else 0.0
            strategies[col] = {'estrategia': 'median', 'valor': val}
        elif pd.api.types.is_datetime64_any_dtype(s):
            m = s.dropna()
            val = m.mode().iloc[0].isoformat() if not m.empty else None
            strategies[col] = {'estrategia': 'mode', 'valor': val}
        elif pd.api.types.is_categorical_dtype(s) or pd.api.types.is_object_dtype(s):
            m = s.dropna().mode()
            val = (m.iloc[0] if len(m) > 0 else 'missing')
            strategies[col] = {'estrategia': 'identificador', 'valor': str(val if pd.notna(val) else 'missing')}
        else:
            strategies[col] = {'estrategia': 'identificador', 'valor': 'missing'}
    return strategies

def save_strategies_json(strategies: dict, path: str | Path = 'imputacion_strategies.json') -> Path:
    path = Path(path)
    with path.open('w', encoding='utf-8') as f:
        json.dump(strategies, f, ensure_ascii=False, indent=2)
    return path

def load_strategies_json(path: str | Path = 'imputacion_strategies.json') -> dict:
    with Path(path).open('r', encoding='utf-8') as f:
        return json.load(f)

def apply_imputation(df: pd.DataFrame, strategies: dict) -> pd.DataFrame:
    out = df.copy()
    for col, spec in strategies.items():
        if col not in out.columns:
            continue
        val = spec.get('valor', None)
        if pd.api.types.is_datetime64_any_dtype(out[col]) and isinstance(val, str):
            fill = pd.to_datetime(val, errors='coerce')
            out[col] = out[col].fillna(fill)
        else:
            out[col] = out[col].fillna(val)
    return out

strategies = build_imputation_strategies(loans_typed)
save_path = save_strategies_json(strategies)
strategies_loaded = load_strategies_json(save_path)
loans_imputed = apply_imputation(loans_typed, strategies_loaded)
loans_imputed.isna().sum().head()


## EDA — limpieza adicional y reproducibilidad
- Quitar columnas inservibles (IDs, URLs, descripciones libres) si existen.
- Mantener funciones para repetir el proceso de forma determinista.


In [ ]:
DROP_COLUMNS = [
    'id', 'member_id', 'url', 'desc', 'title'
]

def drop_unhelpful(df: pd.DataFrame) -> pd.DataFrame:
    cols = [c for c in DROP_COLUMNS if c in df.columns]
    return df.drop(columns=cols)

loans_clean = drop_unhelpful(loans_imputed)
loans_clean.shape


## Pipeline reproducible (función única)
Encapsula lectura → tipos → imputación → drop.


In [ ]:
def etl_pipeline(
    data_url: str = DATA_URL,
    dict_url: str = DICT_URL
):
    loans = read_loans(data_url)
    loans = coerce_types(loans)
    col_types = build_column_types(loans)
    # Diccionario de datos (además de Pickle para cache local)
    dd = pd.read_excel(dict_url)
    dd.columns = ['feature', 'description']
    dd.to_pickle('datos_dict.pkl')
    # Imputación
    strategies = build_imputation_strategies(loans)
    save_strategies_json(strategies, 'imputacion_strategies.json')
    loans = apply_imputation(loans, strategies)
    # Limpieza extra
    loans = drop_unhelpful(loans)
    return loans, col_types, strategies

# Ejecución (opcional)
# loans_final, types_final, strategies_final = etl_pipeline()
# loans_final.head()


## Notas y justificaciones
- Tipos: se prioriza convertir a tipos nativos (`datetime64`, `float`, `int`, `category`) para habilitar análisis eficiente.
- Imputación: mediana en numéricos evita sesgo por outliers; identificador `'missing'` permite trazar valores imputados en categóricas.
- Reproducibilidad: funciones puras y archivo JSON documentan decisiones de limpieza.
